# Spotify API

## Authentication and Authorization Tokens

In [ ]:
import requests
import time

CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"

def get_access_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data, auth=(CLIENT_ID, CLIENT_SECRET))
    token_info = response.json()
    return token_info["access_token"], time.time() + token_info["expires_in"]

access_token, expiry_time = get_access_token()


## Querying

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

print(CLIENT_ID)
print(CLIENT_SECRET)

0101522456894982bd89f75900313bc8
377301b45b1544f08e9d875263d92bfa


## Processing